In [4]:
import pandas as pd 
import matplotlib.pyplot as plt
from math import log10, floor
import re
import os

In [32]:
def round_sig(x, sig=7):
    if x == 0:
        return 0
    elif x <0.1:
        return round(x,sig)
    else: 
        return round(x, sig-int(floor(log10(abs(x))))-1)

def format_number(x):
    if x == 0:
        return '0       '
    else:
        str_x = format(x, '.7f')
        str_x = str_x.lstrip('0')
        if x>1:
            str_x = ' ' + str_x
        return str_x

def create_unsteady_flow_file (template_fp, precip_fp, output_fp, flow_title):
    
    with open(template_fp) as f:
        contents = f.read()

    ### Read in precipitation values which we want to insert into the string
    precip = pd.read_csv(precip_fp)
    precip = precip['Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model']
    precip= precip[0:360]
    precip = precip.apply(round_sig)
    precip = precip.apply(format_number)
    precip = precip.str.cat(sep='')
    precip = re.sub("(.{80})", "\\1\n", precip, 0, re.DOTALL)

    ### Read in the precipitation values from the plan to a list
#     start=contents.find('Precipitation Hydrograph= 360')+len('Precipitation Hydrograph= 360')     
#     end=contents.find('DSS Path') 
#     precip_values = contents[start:end]

    ### Add the new precipitation values to the whole string
#     contents = contents.replace(precip_values,'\n'+ precip)
    contents = contents.replace('precip_values', precip)
    
    # Replace flow title
    if '6h' not in flow_title:
        flow_title = '6h_Cluster{}'.format(flow_title)
#     contents = contents.replace("6h_single-peak", flow_title + '\n')
    contents = contents.replace("flow_title", flow_title)
    
    ### Save to text file
    with open(output_fp, 'w') as f:
        f.write(contents)

def create_plan (plan_title, short_identifier, unsteadyflow_file, output_fp):
    # Read file
    with open('../../../FloodModelling/MeganModel_v1/DissModel.p01') as f:
        contents = f.read()
    if '6h' not in plan_title:
        plan_title = '6h_Cluster{}'.format(plan_title)

    # Replace the variable with the correct values
    contents = contents.replace("6h_single-peak", plan_title)
    contents = contents.replace("6h_sp", short_identifier)
    contents = contents.replace("u01", unsteadyflow_file.split('DissModel.')[1])
    
    ### Save to text file
    with open(output_fp, 'w') as f:
        f.write(contents)

In [33]:
short_ids = ['6h_sp', '6h_dt', '6h_spt', '6h_ms', '6h_c1','6h_c2','6h_c3','6h_c4', '6h_c5', '6h_c6','6h_c7',
             '6h_c8','6h_c9','6h_c10', '6h_c11', '6h_c12','6h_c13','6h_c14','6h_c15',]   
methods = ['6h_single-peak', '6h_divide-time', '6h_subpeak-timing', '6h_max-spread', '1', '2', '3', '4', '5',
           '6', '7', '8', '9','10', '11', '12', '13', '14', '15']  

template_fp =  '../../../FloodModelling/MeganModel_v1/template_unsteadyflowfile.txt'
# template_fp =  '../../../FloodModelling/MeganModel_v3 - Edit/DissModel.u01'

for method_number, method in enumerate(methods, start=1):
    # Read the precipitation data for this method
    print(method)
    if '6h' in method:
        precip_file = "../CreateSyntheticRainfallEvents/MultiplePeaks/PostLossRemoval/6h/{}_urban.csv".format(method)
    else:
        precip_file = "../CreateSyntheticRainfallEvents/RobertoProfiles/PostLossRemoval/6hr_100yrRP/cluster{}_urban_summer.csv".format(method)
    
    # Define filepaths to save the unsteady flow file and plan file to
    if method_number <10:
        unsteadyflow_output_fp = '../../../FloodModelling/MeganModel_v3 - Edit/DissModel.u0{}'.format(method_number)
        plan_output_fp = '../../../FloodModelling/MeganModel_v3 - Edit/DissModel.p0{}'.format(method_number)
    else:
        unsteadyflow_output_fp = '../../../FloodModelling/MeganModel_v3 - Edit/DissModel.u{}'.format(method_number)
        plan_output_fp = '../../../FloodModelling/MeganModel_v3 - Edit/DissModel.p{}'.format(method_number)
        
    # Define the unsteady flow file number (used as an input to the plan)
    unsteadyflow_file_number = 'u0{}'.format(method_number)
    if method != '6h_single-peak':
        # Create unsteady flow and then create the plan
        create_unsteady_flow_file(template_fp, precip_file, unsteadyflow_output_fp, method)
#         create_plan(method, short_ids[method_number-1], unsteadyflow_output_fp, plan_output_fp)


6h_single-peak
6h_divide-time
6h_subpeak-timing
6h_max-spread
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [40]:
import numpy as np
import h5py
os.getcwd()
os.chdir("/nfs/a319/gy17m2a/PhD/FloodModelling/MeganModel_v1 - Copy")

In [45]:
# hf = h5py.File('DissModel.u02.hdf', 'w')
# hf.create_dataset('dataset_1', data=contents)
# hf.close()

f1 = h5py.File('DissModel.u06.hdf','r')    
a_group_key = list(f1.keys())#[0]
f1.keys()
# Print all root level object names (aka keys) 
#     # these can be group or dataset names 
#     print("Keys: %s" % f.keys())
#     # get first object name/key; may or may NOT be a group
#     a_group_key = list(f.keys())[0]

#     # get the object type for a_group_key: usually group or dataset
#     print(type(f[a_group_key])) 

#     # If a_group_key is a group name, 
#     # this gets the object names in the group and returns as a list
#     data = list(f[a_group_key])

#     # If a_group_key is a dataset name, 
#     # this gets the dataset values and returns as a list
#     data = list(f[a_group_key])
#     # preferred methods to get dataset values:
#     ds_obj = f[a_group_key]      # returns as a h5py dataset object
#     ds_arr = f[a_group_key][()]  # returns as a numpy array

<KeysViewHDF5 []>

In [20]:
from pydsstools.heclib.dss import HecDss
import matplotlib.pyplot as plt
import numpy as np

dss_file = "example.dss"
pathname = "/REGULAR/TIMESERIES/FLOW//1HOUR/Ex1/"
startDate = "15JUL2019 19:00:00"
endDate = "15JUL2019 21:00:00"

with HecDss.Open(dss_file) as fid:
    ts = fid.read_ts(pathname,window=(startDate,endDate),trim_missing=True)
    times = np.array(ts.pytimes)

ModuleNotFoundError: No module named 'pydsstools'

In [15]:
os.getcwd()
os.chdir("/nfs/a319/gy17m2a/PhD/FloodModelling/MeganModel_v1 - Copy")

In [17]:
import h5py
def print_h5_structure(f, level=0):
    """    prints structure of hdf5 file    """
    for key in f.keys():
        if isinstance(f[key], h5py._hl.dataset.Dataset):
            print(f"{'  '*level} DATASET: {f[key].name}")
        elif isinstance(f[key], h5py._hl.group.Group):
            print(f"{'  '*level} GROUP: {key, f[key].name}")
            level += 1
            print_h5_structure(f[key], level)
            level -= 1

        if f[key].parent.name == "/":
            print("\n"*2)


file_path = 'DissModel.u06.hdf'
f = h5py.File(file_path, 'r')

print_h5_structure(f)